# Chapter 5) Probabilistic Analysis and Randomized Algorithms

## 5-1) The hiring problem

### 5.1-1
We can model this as a $P \implies Q$ statement. Let us assume $P$, which we'll define as "we are always able to determine which candidate is best", and $Q$ be "we know a total order on the ranks of the candidates". Knowing $P$, we can select any two candidates $a,b$ from the pool of all candidates, and establish a $\le$ relationship between them; $a \le b$. Since it doesn't matter who $a$ or $b$ are, that means any pair of candidates has the $\le$ relationship; thus, by definition of total order, there is a total ordering on candidates.

### 5.1-2
Key points:
* RANDOM(0,1) is akin to a coin flip. We can view the distance between [a,b] as a permutation of coin flips.
* The outcomes of coin flips can be represented using binary
* Representing the number of bits required to represent the distance between [a,b] as $\lceil lg\,(b-a) \rceil$, 
  if we set each bit to the outcome of a call to RANDOM(0,1), we effectively create a random value we can then add to $a$. However, we run the risk of making a value _greater_ than $b-a$. We either need to control for this case, or simply retry until we don't.
  
Questions
* What's the probability of getting a sum that's less than b-a? 

Why can't we just call RANDOM(0,1) b-a+1 times and and that to $a$?
* Since this is like a coin flip, we'd expect to see $\frac{b-a+1}{2}$ heads, the value of which will be disproportionately added to $a$ and returned by such an algorithm. Said another way, there are far more permutations where H and T are equal than there are for all H or T (only 1 each), or all but two H or T ($n \choose 2$ options), etc. 

In [191]:
function random_ab(a, b)
    diff = b-a
    # When diff is a power of two, we need an extra bit to represent
    # its entire range, and floor() makes no difference. When we're not,
    # we need to round up to the next power of two. The 
    m = 1
    while true
        m = 1
        for i = 1:bits
            m = (m << 1) | rand(0:1)
        end
        m -= convert(Int64, 2^bits) # Remove leading bit
        @printf("Value: %d (Binary: %s)\n", m, bin(m))
        if m <= diff
            break
        end
    end
    return a + m
end

random_ab (generic function with 1 method)

__Runtime:__ The for loop is of size $bits$, or $\lfloor lg\,(b-a) \rfloor + 1$, and we have to run it until we get a value less than or equal to $b-a$. If we knew what the probability to generate such a value was, we could simply count on running the algorithm until probability of _not_ generating a satisfactory value was sufficiently small. We would represent this expected repetition as a constant, and eventually end up with $O(lg\,(b-a))$.

But what _is_ the probability of generating a value $\le b-a$? Our procedure inside the `for` loop generates a binary string which can represent $2^{bits}$ possible values, $\le b-a$ of which we are interested in. Thus, we can represent our odds as $\frac{b-a}{2^{bits}}$. There may be something clever here about using the _ordinality_ of the binary string as its _cardinality_, or vice-versa, but I can't quite determine if there is.

Note that our _worst_ odds of getting a satisfactory random value is when $b-a$ is an exact power of two: if $b-a = 2^x$, then we need $2^x+1$ bits to represent it, and end up with $\frac{2^x}{2^{x+1}} = \frac{1}{2}$ as our likelihood. However, we can use $\frac{1}{2}$ in proving for worst-case behvaior, since for any $2^x \lt x \lt 2^{x+1}$ our chances only improve of generating a satisfactory value. To find a constant that guarantees a $\le 1\%$ chance of _not_ generating a unique value, we just need to solve for $x$ in $\frac{1}{2}^x \le .01$, which comes out to be $x \approx 6.64$.

$$
\begin{align}
\frac{1}{2}^x &= .01 \\
2^{-1^x} &= .01 \\
lg\,2^{-x} &= lg\,.01 \\
-x &= -6.643856 \\
x &\approx 6.64
\end{align}
$$
Or alternatively, $x = lg\,\frac{1}{100} = lg\,1 - lg\,100 = -6.64$ for $2^{-6.64}$, as we're looking to get to a one percent chance via division by two.

#ref=https://blog.codinghorror.com/the-danger-of-naivete/

### 5.1-3
Key Insights
* > All you can do is flip the coin multiple times.

    * #ref=https://algox0.wordpress.com/2011/01/31/clrs-5-1-3/
        * ^This guy described the below exp run time using an infinite series.
* With a biased coin, you're equally likely to see [H,T] as you are [T,H]. Thus, you can flip it until you see one of those two combinations, returning the first of the pair as your answer.
* Obviously, substitute H and T for 1 and 0.
* Runtime analysis: $O(\frac{1}{p(1-p)})$
    * Recursive solution: $t = ef + (1-e)(f+t)$
        * $t$ = total number of flips
        * $e$ = Probability of getting HT or TH
        * $f$ = # of flips per round; 2, in our case.
        * $ef$ = Probability of getting a "bit", a.k.a a 01 or a 10.
        * $(1-e)$ = Weight for _not_ getting a bit
        * $(f+t)$ = Add $f$ flips to the total, weighted by _not_ getting a bit.
    * Simplifies to $t = \frac{f}{e}$
        * $t = \frac{2}{2p(1-p)} = \frac{1}{p(1-p)}$

#verified=http://www.eecs.harvard.edu/~michaelm/coinflipext.pdf

# 5-2) Indicator Random Variables
Let's flip a coin. Say we represent the event we get heads with $X_H$. We can use an _indicator random variable_ to represent the outcomes of this event numerically; 1 if we get heads, 0 otherwise (presumably tails; what kind of coin flippin' are you doing?): $I\{H\}$. We translate _expected value_ of our event into the probability that of each outcome: $E[X_H] = E[I\{H\}] = 1\cdot Pr\{H\} + 0\cdot Pr\{T\}$.

### 5.2-1) 
If we hire only once, then the first candidate was also the best. There's a $\frac{1}{n}$ likelihood of their being in the first position, so $Pr\{H_{Once}\} = \frac{1}{n}$

If we hire _every_ candidate (and subsequently fire all but the last), then the candidates arrived in perfectly ascending order of their score. As stated, there's a $\frac{1}{n}$ chance of any candidate being in any position, so $Pr\{H_n\} = \frac{1}{1} \cdot \frac{1}{2} \cdot \ldots \cdot \frac{1}{n-1} \cdot \frac{1}{n} = \frac{1}{n!}$.

#verified=https://mitpress.mit.edu/sites/default/files/titles/content/Intro_to_Algo_Selected_Solutions.pdf

5.2-2)

#ref=http://stackoverflow.com/a/21812040/2246784
#ref=http://m.blog.csdn.net/article/details?id=38979351
#ref=https://cyberzhg.gitbooks.io/clrs/content/Chapter_05_Probabilistic_Analysis_and_Randomized_Algorithms/exercises_5.2.html

## 5-3) Randomized Algorithms
Whats the problem: Our runtime complexity is strongly affected by the kinds of input we receive, in particular, the ordering of their contents.

Why does this matter: We can only provide average-case runtime guarantees when we can make strong guarantees about the kinds of input.

HW Problems: All concerned with proving uniform random permutations are generated. Thus, proving the likelihood that any given item $A[i]$ appears anywhere with a uniform probability.

Questions: 
* How do you determine the probability an item appears in a given location? 
* How do you know something is uniformly random? 
    * When its probability of appearing in a location is the same for all locations
    
Key Concepts
* Randomized algorithms impose a random distribution of inputs
    * Probability of a given ordering = $Pr\{E_1 \cap E_2\} = Pr\{E_1\mid E_2\}Pr\{E_2\}$
    * Probability of a random _n_-permutation = $\frac{1}{n}$
* Uniform random permutations (URPs)
    * $\frac{1}{n!}$
* You can produce a URP by 
    1. Assigning random rankings to each element, then sorting
    2. Swapping each element with itself or any following element

### 5.3-1
Key points
* Swap the first element _before_ entering the loop
* Start the loop at $i=2$.
* With our new base case of $i=2$, $\frac{(n-2+1)!}{n!} = \frac{(n-1)!}{n!} = \frac{1}{n}$. English: We know choosing randomly from 1 to $n$ numbers has a $\frac{1}{n}$ probability of choosing any one number, thus all 1-permutations are equally likely.

```
RandomizeInPlace(A)
n = A.length
SWAP(A[1], A[RANDOM(1, n)])
for i=2 to n
    SWAP(A[i], A[RANDOM(i,n)])
```
Base Case Proof (i=2): Each 1-permutation must have equal likelihood. Since $\frac{(n-2+1)!}{n!} = \frac{(n-1)!}{n!} = \frac{1}{n}$, we see this to be true.



#verified=https://courses.csail.mit.edu/6.046/fall01/handouts/ps2sol.pdf

### 5.4) Probabilistic analysis and further uses of indicator random variables

##### Before
Describe the problem) 

Why is this a problem?)

**Action: Skim material** 

Theme of HW Problems)

Initial Questions)

##### During
**Action: Start timer**

Key Concepts) 

Relationships)

Notes) 

Questions)

Unknowns)

##### After
**Action: Stop timer**

Review Questions)

Feynman)